# インポート

In [4]:
import pandas as pd
from pathlib import Path

COMMON_DATA_DIR = Path("..", "..", "common", "data")
RAWDF_DIR = COMMON_DATA_DIR / "rawdf"

# データ加工

## レース結果テーブルの加工

In [6]:
# レース結果のCSVを読み込み
df = pd.read_csv(RAWDF_DIR / "results.csv", sep="\t")

In [ ]:
# すべて表示すると見にくいため、表示するデータを絞る
# iloc[行, 列]で指定できる。「:」はすべてを表す。
# よって次の記述はcsvの全行かつ先頭から10列分を表示。
df.iloc[:, 0:10]

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差
0,202408060405,1,8,10,エコロディノス,牡2,56.0,ムーア,2:00.9,NaN
1,202408060405,2,5,5,トロワヴァレー,牡2,56.0,Ｃ．デム,2:01.5,3.1/2
2,202408060405,3,1,1,コジオスコ,牡2,56.0,ルメール,2:01.9,2.1/2
3,202408060405,4,7,8,アロマフェリス,牝2,55.0,鮫島克駿,2:02.4,3
4,202408060405,5,8,9,サウスオブロンドン,牡2,55.0,泉谷楓真,2:02.7,2
...,...,...,...,...,...,...,...,...,...,...
46544,202405010107,12,3,5,グランヒマラヤ,牡4,57.0,石橋脩,1:29.1,2.1/2
46545,202405010107,13,4,8,フエキ,牝5,56.0,嶋田純次,1:29.2,クビ
46546,202405010107,14,6,12,ジャーニーメモリー,牡4,56.0,原優介,1:29.2,クビ
46547,202405010107,15,8,15,テンプーシャオン,牡4,57.0,大野拓弥,1:30.4,7


In [8]:
# 着順列を取得
df["着順"]

0         1
1         2
2         3
3         4
4         5
         ..
46544    12
46545    13
46546    14
46547    15
46548    16
Name: 着順, Length: 46549, dtype: object

In [9]:
# 着順列のdtypeがobjectになっている
# 機械学習モデルに入れるにはobjectだと入れられないため
# 数値型に変換する
df["着順"].astype(int)

ValueError: invalid literal for int() with base 10: '中'

In [ ]:
# エラーになったのは着順列に、中止を表す「中」が含まれているためint変換できない
# 数値でない値は省く
# pandasのto_numericを使えば、エラー時の動きを指定できる
# デフォルトはraiseでエラーを返す、coerceを指定すればエラーになるものは欠損値に変換する
pd.to_numeric(df["着順"], errors="coerce")

0         1.0
1         2.0
2         3.0
3         4.0
4         5.0
         ... 
46544    12.0
46545    13.0
46546    14.0
46547    15.0
46548    16.0
Name: 着順, Length: 46549, dtype: float64

In [11]:
# 欠損値がどれくらいあるのか調べたい
df["rank"] = pd.to_numeric(df["着順"], errors="coerce")
# value_countsはデフォルトでは欠損値が省かれるため、dropna=Falseを指定する
df["rank"].value_counts(dropna=False)

rank
1.0     3409
3.0     3405
2.0     3404
4.0     3404
5.0     3399
6.0     3395
7.0     3375
8.0     3285
9.0     3177
10.0    3009
11.0    2785
12.0    2520
13.0    2246
14.0    1991
15.0    1625
16.0    1203
NaN      422
17.0     292
18.0     203
Name: count, dtype: int64

In [13]:
# 目的変数でrankを求める際に、NaNのデータは不要なので省いてあげる
# rankの中から欠損値を省く処理、inplace=Trueはdf自体に変更を加える
df.dropna(subset=["rank"], inplace=True)

In [14]:
df["rank"].value_counts(dropna=False)

rank
1.0     3409
3.0     3405
2.0     3404
4.0     3404
5.0     3399
6.0     3395
7.0     3375
8.0     3285
9.0     3177
10.0    3009
11.0    2785
12.0    2520
13.0    2246
14.0    1991
15.0    1625
16.0    1203
17.0     292
18.0     203
Name: count, dtype: int64

In [15]:
# 枠番、馬番
df["枠番"].astype(int)

0        8
1        5
2        1
3        7
4        8
        ..
46544    3
46545    4
46546    6
46547    8
46548    2
Name: 枠番, Length: 46127, dtype: int64

In [ ]:
# 性齢は、性別と年齢がくっついているので分けてあげる
df["性齢"]

0        牡2
1        牡2
2        牡2
3        牝2
4        牡2
         ..
46544    牡4
46545    牝5
46546    牡4
46547    牡4
46548    牡6
Name: 性齢, Length: 46127, dtype: object

In [ ]:
# .str[0]で１文字を取得
df["性齢"].str[0].value_counts()

性齢
牡    24843
牝    19150
セ     2134
Name: count, dtype: int64

ラベルエンコーディング：カテゴリ変数を整数にマッピングするデータ加工手法

In [ ]:
# 性別が文字なので数値にする必要がある
# ラベルエンコーディングという手法を使う
sex_mapping = {"牡": 0, "牝": 1, "セ": 2}

In [22]:
# これでマッピングできる
df["性齢"].str[0].map(sex_mapping)

0        0
1        0
2        0
3        1
4        0
        ..
46544    0
46545    1
46546    0
46547    0
46548    0
Name: 性齢, Length: 46127, dtype: int64